In [ ]:
pip install wikipedia

In [ ]:
pip install -U bitsandbytes

In [ ]:
!pip install wikipedia

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datetime import datetime
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import wikipedia
import math, re

In [ ]:
pip install -U bitsandbytes transformers accelerate


In [ ]:

model_name = "tiiuae/falcon-7b-instruct"  # or another instruct model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:

_ALLOWED_RE = re.compile(r"^[\d\s\+\-\*\/\%\.\(\)\^]*$")
def calculator_tool(task):
    expr = task.strip().replace("^", "**")
    if not _ALLOWED_RE.match(expr):
        return "Error"
    try:
        return str(eval(expr, {"__builtins__": None}, math.__dict__))
    except Exception:
        return "Error"

def datetime_tool(task):
    now = datetime.now()
    task = task.lower()
    if "date" in task:
        return now.strftime("%Y-%m-%d")
    elif "time" in task:
        return now.strftime("%H:%M:%S")
    return "Please specify 'date' or 'time'."

# ---- Weather (mock example) ----
def weather_tool(city):
    data = {
        "New York": "10°C, cloudy",
        "London": "12°C, light rain",
        "Delhi": "28°C, clear",
        "Tokyo": "18°C, windy"
    }
    return f"The weather in {city} is {data.get(city, 'unavailable')}."

# ---- Wikipedia ----
def wikipedia_tool(query):
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception:
        return "No result found."

# ---- Plot ----
def plot_tool(expression, start=-10, end=10):
    try:
        x = np.linspace(start, end, 400)
        y = eval(expression, {"x": x, "np": np, "__builtins__": None})
        plt.figure(figsize=(6,4))
        plt.plot(x, y, label=f"y = {expression}")
        plt.title(f"Plot of {expression}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        plt.grid(True)
        plt.savefig("plot_result.png")
        plt.close()
        display(Image("plot_result.png"))
        return "✅ Plot generated and displayed below."
    except Exception as e:
        return f"Error plotting: {e}"



def tool_router(response_text):
    """
    Detects and executes tool requests.
    Returns tool name + result.
    """
    tool_patterns = {
        "CALCULATOR": calculator_tool,
        "DATETIME": datetime_tool,
        "WEATHER": weather_tool,
        "WIKIPEDIA": wikipedia_tool,
        "PLOT": plot_tool
    }

    m = re.search(r"USE_TOOL:\s*([A-Z]+)\s*(.*)", response_text.strip())
    if not m:
        return None

    tool_name, argument = m.groups()
    tool_func = tool_patterns.get(tool_name)

    if tool_func:
        print(f"[🧩 Tool Invoked: {tool_name}] with argument → {argument.strip()}")
        result = tool_func(argument.strip())
        return {"tool": tool_name, "result": result, "argument": argument.strip()}
    return None



In [ ]:


def run_agent(pipe, user_query):
    """
    Runs query through model -> optional tool -> reasoning with tool result.
    """
    base_prompt = f"""
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: {user_query}
"""
    # Step 1: Model decides how to respond
    response = pipe(base_prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]
    print("\n[🤖 Model Output]\n", response)

    # Step 2: If model asks to use a tool
    tool_result = tool_router(response)
    if tool_result:
        print(f"[🔧 Tool Result]: {tool_result['result']}")

        # Step 3: Re-prompt model to interpret tool output
        reasoning_prompt = f"""
The user asked: {user_query}
You used the {tool_result['tool']} tool with input: {tool_result['argument']}
The tool returned: {tool_result['result']}

Now, using this result, provide a clear, concise explanation to the user:
"""
        final_response = pipe(reasoning_prompt, max_new_tokens=150, do_sample=False,return_full_text=False)[0]["generated_text"]
        print("\n[🧠 Final Reasoned Answer]\n", final_response)

    else:
        # Direct text answer (no tool needed)
        print("[Direct Answer]\n", response)



In [ ]:
run_agent(pipe, "What is 15 * (8 + 2)?")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: What is 15 * (8 + 2)?
Answer: 120
[Direct Answer]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: What is 15 * (8 + 2)?
Answer: 120


In [ ]:
run_agent(pipe, "What time is it now?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: What time is it now?
Answer:
It is currently <current_time> (e.g., 12:30 PM).
[Direct Answer]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: What time is it now?
Answer:
It is currently <current_time> (e.g., 12:30 PM).


In [ ]:
run_agent(pipe, "What's the weather in London?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: What's the weather in London?
Answer:
London has a temperate maritime climate, with mild summers and cool winters. The city experiences frequent rainfall, with an average of 1,000mm per year.
[Direct Answer]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: What's the weather in London?
Answer:
London has a temperate maritime climate, w

In [ ]:

run_agent(pipe, "Tell me about Albert Einstein.")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: Tell me about Albert Einstein.
Answer: Albert Einstein was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics. His work is also known for its influence on the philosophy of science. He is best known for his mass-energy equivalence formula E=mc^2, which relates energy and mass. He received the Nobel Prize in Physics in 1921.
[Direct Answer]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions 

In [ ]:
run_agent(pipe, "Plot y = x**2 - 4*x + 3 and explain what it represents.")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions (e.g., x**2)

If you need a tool, respond with:
"USE_TOOL: <TOOL_NAME> <query>"

Otherwise, answer directly.

Question: Plot y = x**2 - 4*x + 3 and explain what it represents.
Answer: The plot represents the function y = x**2 - 4*x + 3. The x-axis represents the variable x, and the y-axis represents the variable y. The plot shows the value of y for different values of x. In this case, the plot is a parabola that is shifted to the left by 4 units of x. The parabola is also shifted up by 3 units of y, which means that the value of y
[Direct Answer]
 
You are an AI assistant with access to these tools:
- CALCULATOR: for math (e.g., 2+2)
- DATETIME: for date/time queries
- WEATHER: for city weather info
- WIKIPEDIA: for factual summaries
- PLOT: to 

Final Version: Multi-Turn, Context-Aware Tool Agent

In [ ]:
# ============================================================
# 3. Tool Implementations
# ============================================================

# ---- Calculator (secure) ----
_ALLOWED_RE = re.compile(r"^[\d\s\+\-\*\/\%\.\(\)\^]*$")
def calculator_tool(task):
    expr = task.strip().replace("^", "**")
    if not _ALLOWED_RE.match(expr):
        return "Error"
    try:
        return str(eval(expr, {"__builtins__": None}, math.__dict__))
    except Exception:
        return "Error"

# ---- Datetime ----
def datetime_tool(task):
    now = datetime.now()
    task = task.lower()
    if "date" in task:
        return now.strftime("%Y-%m-%d")
    elif "time" in task:
        return now.strftime("%H:%M:%S")
    return "Please specify 'date' or 'time'."

# ---- Weather (mock) ----
def weather_tool(city):
    data = {
        "New York": "10°C, cloudy",
        "London": "12°C, light rain",
        "Delhi": "28°C, clear",
        "Tokyo": "18°C, windy"
    }
    return f"The weather in {city} is {data.get(city, 'unavailable')}."

# ---- Wikipedia ----
def wikipedia_tool(query):
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception:
        return "No result found."

# ---- Plot ----
def plot_tool(expression, start=-10, end=10):
    try:
        x = np.linspace(start, end, 400)
        y = eval(expression, {"x": x, "np": np, "__builtins__": None})
        plt.figure(figsize=(6,4))
        plt.plot(x, y, label=f"y = {expression}")
        plt.title(f"Plot of {expression}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        plt.grid(True)
        plt.savefig("plot_result.png")
        plt.close()
        display(Image("plot_result.png"))
        return "✅ Plot generated and displayed below."
    except Exception as e:
        return f"Error plotting: {e}"

# ============================================================
# 4. Tool Router
# ============================================================

def tool_router(response_text):
    tool_patterns = {
        "CALCULATOR": calculator_tool,
        "DATETIME": datetime_tool,
        "WEATHER": weather_tool,
        "WIKIPEDIA": wikipedia_tool,
        "PLOT": plot_tool
    }

    m = re.search(r"USE_TOOL:\s*([A-Z]+)\s*(.*)", response_text.strip())
    if not m:
        return None

    tool_name, argument = m.groups()
    tool_func = tool_patterns.get(tool_name)

    if tool_func:
        print(f"[🧩 Tool Invoked: {tool_name}] with argument → {argument.strip()}")
        result = tool_func(argument.strip())
        return {"tool": tool_name, "result": result, "argument": argument.strip()}
    return None


In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datetime import datetime
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import wikipedia
import math, re

# # ============================================================
# # 2. Initialize Model Pipeline
# # ============================================================
# model_name = "tiiuae/falcon-7b-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     load_in_8bit=True
# )
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


# ============================================================
# 5. Agent Class with Memory
# ============================================================
import json
import os

class MemoryAgent:
    def __init__(self, pipe, memory_file="agent_memory.json"):
        self.pipe = pipe
        self.memory_file = memory_file
        self.memory = self._load_memory()

    # ------------------------------
    # Helper functions
    # ------------------------------
    def _load_memory(self):
        if os.path.exists(self.memory_file):
            try:
                with open(self.memory_file, "r") as f:
                    return json.load(f)
            except Exception:
                return []
        return []

    def _save_memory(self):
        with open(self.memory_file, "w") as f:
            json.dump(self.memory, f, indent=4)

    def _generate(self, prompt, max_tokens=150):
        return self.pipe(prompt, max_new_tokens=max_tokens, do_sample=False)[0]["generated_text"]

    # ------------------------------
    # Main run method
    # ------------------------------
    def run(self, user_query):
        # Only include recent 3 interactions for context (short + clean)
        last_memories = self.memory[-3:]
        memory_context = "\n".join([
            f"User asked: {m['query']}\nTool: {m['tool']}\nResult: {m['result']}"
            for m in last_memories
        ])

        prompt = f"""
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
{memory_context}

New question: {user_query}

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.
"""
        # Step 1: Model output
        response = self._generate(prompt)
        print("\n[🤖 Model Output]\n", response)

        # Step 2: Tool routing
        tool_result = tool_router(response)
        if tool_result:
            print(f"[🔧 Tool Result]: {tool_result['result']}")

            # Step 3: Reasoning
            reasoning_prompt = f"""
User asked: {user_query}
You used the {tool_result['tool']} tool with input: {tool_result['argument']}
The tool returned: {tool_result['result']}

Now explain the result clearly to the user.
"""
            final_response = self._generate(reasoning_prompt)
            print("\n[🧠 Final Reasoned Answer]\n", final_response)

            # Step 4: Save to JSON memory
            self.memory.append({
                "query": user_query,
                "tool": tool_result['tool'],
                "argument": tool_result['argument'],
                "result": tool_result['result'],
                "explanation": final_response.strip()
            })
            self._save_memory()

        else:
            print("[Direct Answer]\n", response)
            self.memory.append({
                "query": user_query,
                "tool": None,
                "result": None,
                "explanation": response.strip()
            })
            self._save_memory()



In [ ]:

agent = MemoryAgent(pipe)


In [ ]:


agent.run("What is 10 * (5 + 2)?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):


New question: What is 10 * (5 + 2)?

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.

10 * (5 + 2) = 10 * 7 = 70.
[Direct Answer]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):


New question: What is 10 * (5 + 2)?

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.

10 * (5 + 2) = 10 * 7 = 70.


In [ ]:


agent.run("Plot y = x**2 - 4*x + 3")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: What is 10 * (5 + 2)?
Tool: None
Result: None

New question: Plot y = x**2 - 4*x + 3

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.
User 
[Direct Answer]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: What is 10 * (5 + 2)?
Tool: None
Result: None

New question: Plot y = x**2 - 4*x + 3

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.
User 


In [ ]:


agent.run("Now plot the derivative of the previous function.")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: What is 10 * (5 + 2)?
Tool: None
Result: None
User asked: Plot y = x**2 - 4*x + 3
Tool: None
Result: None

New question: Now plot the derivative of the previous function.

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.
User 
[Direct Answer]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: What is 10 * (5 + 2)?
Tool: None
Result: None
User asked: Plot y = x**2 - 4*x + 3
Tool: None
Result: None

New question: Now plot the derivative of the previous function.

If a tool is needed, resp

In [ ]:


agent.run("Who is Albert Einstein?")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: What is 10 * (5 + 2)?
Tool: None
Result: None
User asked: Plot y = x**2 - 4*x + 3
Tool: None
Result: None
User asked: Now plot the derivative of the previous function.
Tool: None
Result: None

New question: Who is Albert Einstein?

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.
User 
[Direct Answer]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: What is 10 * (5 + 2)?
Tool: None
Result: None
User asked: Plot y = x**2 - 4*x + 3
Tool: None
Result: None
User asked: Now plot the deriva

In [ ]:

agent.run("Summarize the previous answer in one sentence.")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



[🤖 Model Output]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: Plot y = x**2 - 4*x + 3
Tool: None
Result: None
User asked: Now plot the derivative of the previous function.
Tool: None
Result: None
User asked: Who is Albert Einstein?
Tool: None
Result: None

New question: Summarize the previous answer in one sentence.

If a tool is needed, respond with:
"USE_TOOL: <TOOL_NAME> <query>"
Otherwise, answer directly.
User 
[Direct Answer]
 
You are an intelligent AI assistant with these tools:
- CALCULATOR: for math
- DATETIME: for date/time
- WEATHER: for weather info
- WIKIPEDIA: for factual summaries
- PLOT: to visualize math functions

Context (last 3 interactions):
User asked: Plot y = x**2 - 4*x + 3
Tool: None
Result: None
User asked: Now plot the derivative of the previous function.
To